In [1]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os
import cv2
import csv
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
original_data_dir = 'images'
final_data_dir = 'final_data'
train_dir = os.path.join(final_data_dir, 'Train_Nums')
test_dir = os.path.join(final_data_dir, 'Test_Nums')

# Create final_data and its subdirectories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to create subdirectories for digits and unknown
def create_subdirs(base_dir):
    for i in range(10):
        os.makedirs(os.path.join(base_dir, str(i)), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'unknown'), exist_ok=True)

# Create subdirectories for train and test
create_subdirs(train_dir)
create_subdirs(test_dir)

# Function to split and copy files
def split_and_copy_files(class_dir):
    # List all files in the class directory
    files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
    # Split files into train and test sets
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    return train_files, test_files

# Traverse the original data directory
for class_name in os.listdir(original_data_dir):
    class_dir = os.path.join(original_data_dir, class_name)
    if os.path.isdir(class_dir) and class_name.isdigit() or class_name == 'unknown':
        # Split files
        train_files, test_files = split_and_copy_files(class_dir)
        # Copy train files
        for file_name in train_files:
            shutil.copy(os.path.join(class_dir, file_name), os.path.join(train_dir, class_name, file_name))
        # Copy test files
        for file_name in test_files:
            shutil.copy(os.path.join(class_dir, file_name), os.path.join(test_dir, class_name, file_name))

print("Train-test split completed successfully.")


Train-test split completed successfully.


In [39]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths
train_dir = 'final_data/Train_Nums'
test_dir = 'final_data/Test_Nums'

# Image dimensions
IMG_HEIGHT, IMG_WIDTH = 100, 100
BATCH_SIZE = 32

# ImageDataGenerators for loading images
train_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

test_data_gen = test_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)


Found 13348 images belonging to 11 classes.
Found 3342 images belonging to 11 classes.


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(11, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 128)            │     1,638,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,733,195 (6.61 MB)

 Trainable params: 1,733,195 (6.61 MB)

 Non-trainable params: 0 (0.00 B)

In [46]:
EPOCHS = 1

history = model.fit(
    train_data_gen,
    epochs=EPOCHS,
    validation_data=test_data_gen
)


418/418 ━━━━━━━━━━━━━━━━━━━━ 36s 86ms/step - accuracy: 0.9797 - loss: 0.0550 - val_accuracy: 0.9647 - val_loss: 0.1717


In [47]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_data_gen)
print('\nTest accuracy:', test_acc)


105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.9672 - loss: 0.1445

Test accuracy: 0.9646918177604675


In [48]:
model.save('model/sign_language_model.h5')

In [50]:
model_json = model.to_json()
with open("model/sign_language_model.json",'w') as json_file:
    json_file.write(model_json)

In [49]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load the saved model
model = load_model('model/sign_language_model.h5')

# Class labels (update this to match your actual class labels)
class_labels = list(train_data_gen.class_indices.keys())

# Function to capture and predict from webcam
def predict_from_webcam(model, class_labels):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the image
        img = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
        img = np.expand_dims(img, axis=0)
        img = img / 255.0

        # Predict the class
        prediction = model.predict(img)
        class_idx = np.argmax(prediction)
        predicted_class = class_labels[class_idx]

        # Display the prediction
        cv2.putText(frame, f'Prediction: {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('Sign Language Number Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run real-time prediction
predict_from_webcam(model, class_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━